In [1]:
from train_arcface import train_arcface
from src.utils.model import get_triplet_model, freeze_layer
from src.model.arcface import KSubArcFace, ArcFace_org
import torch
from train_triplet import train_triplet
from train_arc_triplet import train_arc_triplet

In [2]:
WEIGHT_PATH = 'weight/backbone_r100_glint360k.pt'
TRAIN_PATH = "Data/Data_train" 
TEST_PATH = "Data/Data_test"
EPOCHS = 1000
BATCH__SIZE = 16

# FINETUNE_PATH = 'weight/model_finetune_arcface_org_best.pth'
# ARCFACE_PATH = 'weight/arcface_org_weight_best.pth'

In [ ]:
model = get_triplet_model(weight_path= WEIGHT_PATH)
# model.backbone.load_state_dict(torch.load(FINETUNE_PATH, map_location = 'cpu')['model_state_dict'])

arcface = ArcFace_org(in_features = 512, out_features = 670)
# arcface.load_state_dict(torch.load(ARCFACE_PATH, map_location = 'cpu')['model_state_dict'])


train_arc_triplet(model, arcface, TRAIN_PATH, TEST_PATH, EPOCHS, BATCH__SIZE, roc_fig_path='result/arcface_loss_1000+200_epoches.jpg' )

In [ ]:
a,p,n,_,_,_ = model()

In [ ]:
# model = get_triplet_model(weight_path= WEIGHT_PATH)
# freeze_layer(model, ['253', '254'])
# train_triplet(model, TRAIN_PATH, TEST_PATH, EPOCHS, BATCH__SIZE, roc_fig_path='result/finetune_triplet_loss.jpg')